### Response Completeness Evaluator

### Getting Started

This sample demonstrates how to use Response Completeness Evaluator on agent's response when ground truth is provided. This evaluator is helpful when you have ground truth to assess the quality of the agent's final response. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name


The Response Completeness evaluator assesses the quality of an agent response by examining how well it aligns with the provided ground truth. The evaluation is based on the following scoring system:

<pre>
Score 1: Fully incomplete: The response misses all necessary and relevant information compared to the ground truth.
Score 2: Barely complete: The response contains only a small percentage of the necessary information.
Score 3: Moderately complete: The response includes about half of the necessary information.
Score 4: Mostly complete: The response contains most of the necessary information, with only minor omissions.
Score 5: Fully complete: The response perfectly matches all necessary and relevant information from the ground truth.
</pre>

The evaluation requires the following inputs:

- Response: The response to be evaluated. (string)
- Ground Truth: The correct and complete information against which the response is compared. (string)

The evaluator uses these inputs to determine the completeness score, ensuring that the response meaningfully addresses the query while adhering to the provided definitions and data.

In [1]:
%pip install azure-ai-projects azure-identity azure-ai-evaluation python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Initialize Completeness Evaluator


In [3]:
from azure.ai.evaluation import ResponseCompletenessEvaluator, AzureOpenAIModelConfiguration
from pprint import pprint
import os

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

In [4]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

response_completeness_evaluator = ResponseCompletenessEvaluator(model_config=model_config)

Class ResponseCompletenessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Samples

#### Evaluating for a ground_truth and response

In [5]:
# agent response is complete
result = response_completeness_evaluator(
    response="Itinery: Day 1 check out the downtown district of the city on train; for Day 2, we can rest in hotel.",
    ground_truth="Itinery: Day 1 take a train to visit the downtown area for city sightseeing; Day 2 rests in hotel.",
)
result

{'response_completeness': 4,
 'response_completeness_result': 'pass',
 'response_completeness_threshold': 3,
 'response_completeness_reason': 'The response is mostly complete as it includes most of the relevant information but omits a minor detail about "city sightseeing" for Day 1.'}

In [6]:
# agent response is incomplete
result = response_completeness_evaluator(
    response="The order with ID 123 has been shipped and is expected to be delivered on March 15, 2025. However, the order with ID 124 is delayed and should now arrive by March 20, 2025.",
    ground_truth="The order with ID 124 is delayed and should now arrive by March 20, 2025.",
)
result

{'response_completeness': 5,
 'response_completeness_result': 'pass',
 'response_completeness_threshold': 3,
 'response_completeness_reason': 'The response accurately includes all the relevant information from the ground truth without missing any details, making it fully complete.'}

### Prepare ground truth for agent response


In [7]:
import json
from pathlib import Path

data = [
    {
        "response": "The temperature of Seattle now is 70 degrees. Based on the temperature, having an outdoor office party is recommended.",
        "ground_truth": "The temperature of Seattle now is 50 degrees. It will be recommended to bring a jacket in the evening.",
    },
    {
        "response": 'The email draft "Project Plan" is attached. Please review and provide feedback.',
        "ground_truth": 'The email draft "Project Plan" is attached. Please review and provide feedback by EOD.',
    },
    {
        "response": "Based on the retrieved documents, the shareholder meeting discussed the operational efficiency of the company and financing options.",
        "ground_truth": "The shareholder meeting discussed the compensation package of the company CEO.",
    },
    {
        "response": "The calendar API returns an error code 500. Please check the server logs for more details.",
        "ground_truth": "The meeting is scheduled for 2 PM tomorrow. Please confirm your availability by EOD.",
    },
]

file_path = "response_completeness_data.jsonl"

with Path.open(file_path, "w") as file:
    for line in data:
        file.write(json.dumps(line) + "\n")

## Batch evaluate and visualize results on Azure AI Foundry
Batch evaluate to leverage asynchronous evaluation on a dataset. 

Optionally, you can go to AI Foundry URL for rich Azure AI Foundry data visualization. You can inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve. Make sure to authenticate to Azure using `az login` in your terminal before running this cell.

In [8]:
from azure.ai.evaluation import evaluate

azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "project_name": os.environ["PROJECT_NAME"],
    "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
}

response = evaluate(
    data=file_path,
    evaluators={
        "response_completeness": response_completeness_evaluator,
    },
    azure_ai_project={
        "subscription_id": os.environ["REPORT_AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["REPORT_PROJECT_NAME"],
        "resource_group_name": os.environ["REPORT_RESOURCE_GROUP_NAME"],
    },
)

pprint(f'AI Foundry URL: {response.get("studio_url")}')

[2025-06-04 14:27:27 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-04 14:27:27 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_response_completeness_20250604_142727_724371, log path: /Users/cv/.promptflow/.runs/azure_ai_evaluation_evaluators_response_completeness_20250604_142727_724371/logs.txt


2025-06-04 14:27:27 -0700   82136 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Finished 1 / 4 lines.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Average execution time for completed lines: 2.32 seconds. Estimated time for incomplete lines: 6.96 seconds.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Finished 2 / 4 lines.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Average execution time for completed lines: 1.17 seconds. Estimated time for incomplete lines: 2.34 seconds.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Finished 3 / 4 lines.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Average execution time for completed lines: 0.79 seconds. Estimated time for incomplete lines: 0.79 seconds.
2025-06-04 14:27:30 -0700   82136 execution.bulk     INFO     Finished 4 / 4 lines.
2025-

In [9]:
import pandas as pd
pd.DataFrame(response["rows"])

,inputs.response,inputs.ground_truth,outputs.response_completeness.response_completeness,outputs.response_completeness.response_completeness_result,outputs.response_completeness.response_completeness_threshold,outputs.response_completeness.response_completeness_reason,line_number
0,The temperature of Seattle now is 70 degrees. ...,The temperature of Seattle now is 50 degrees. ...,1,fail,3,The response is fully incomplete because it do...,0
1,"The email draft ""Project Plan"" is attached. Pl...","The email draft ""Project Plan"" is attached. Pl...",4,pass,3,The response is mostly complete but misses the...,1
2,"Based on the retrieved documents, the sharehol...",The shareholder meeting discussed the compensa...,1,fail,3,The response completely misses the necessary a...,2
3,The calendar API returns an error code 500. Pl...,The meeting is scheduled for 2 PM tomorrow. Pl...,1,fail,3,The response does not contain any of the infor...,3
